# Reinforcement Learning Solution to the Towers of Hanoi Puzzle

For this assignment, you will use reinforcement learning to solve the [Towers of Hanoi](https://en.wikipedia.org/wiki/Tower_of_Hanoi) puzzle.  

To accomplish this, you must modify the code discussed in lecture for learning to play Tic-Tac-Toe.  Modify the code  so that it learns to solve the three-disk, three-peg
Towers of Hanoi Puzzle.  In some ways, this will be simpler than the
Tic-Tac-Toe code.  

Steps required to do this include the following:

  - Represent the state, and use it as a tuple as a key to the Q dictionary.
  - Make sure only valid moves are tried from each state.
  - Assign reinforcement of $-1$ to each move unless it is a move to the goal state, for which the reinforcement is $0$.  This represents the goal of finding the shortest path to the goal.

Make a plot of the number of steps required to reach the goal for each
trial.  Each trial starts from the same initial state.  Decay epsilon
as in the Tic-Tac-Toe code.

## Requirements

First, how should we represent the state of this puzzle?  We need to keep track of which disks are on which pegs. Name the disks 1, 2, and 3, with 1 being the smallest disk and 3 being the largest. The set of disks on a peg can be represented as a list of integers.  Then the state can be a list of three lists.

For example, the starting state with all disks being on the left peg would be `[[1, 2, 3], [], []]`.  After moving disk 1 to peg 2, we have `[[2, 3], [1], []]`.

To represent that move we just made, we can use a list of two peg numbers, like `[1, 2]`, representing a move of the top disk on peg 1 to peg 2.

Now on to some functions. Define at least the following functions. Examples showing required output appear below.

   - `printState(state)`: prints the state in the form shown below
   - `validMoves(state)`: returns list of moves that are valid from `state`
   - `makeMove(state, move)`: returns new (copy of) state after move has been applied.
   - `trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF)`: train the Q function for number of repetitions, decaying epsilon at start of each repetition. Returns Q and list or array of number of steps to reach goal for each repetition.
   - `testQ(Q, maxSteps, validMovesF, makeMoveF)`: without updating Q, use Q to find greedy action each step until goal is found. Return path of states.

A function that you might choose to implement is

   - `stateMoveTuple(state, move)`: returns tuple of state and move.  
    
This is useful for converting state and move to a key to be used for the Q dictionary.

In [493]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import copy

In [494]:
def printState(state):
    string = ""
    if state[0] and state[1] and state[2]:
        string = string + str(state[0][0]) + " " +str(state[1][0]) + " "+str(state[2][0]) + "\n"
    if state[0] and not state[1] and not state[2]:
        string = string + str(state[0][0]) + "\n" +str(state[0][1]) + "\n"+str(state[0][2]) + "\n"
    if not state[0] and state[1] and not state[2]:
        string = string + "  " + str(state[1][0]) + "\n" + "  " +str(state[1][1]) + "\n" + "  " + str(state[1][2]) + "\n"
    if not state[0] and not state[1] and state[2]:
        string = string + "    " + str(state[2][0]) + "\n" + "    " +str(state[2][1]) + "\n" + "    " + str(state[2][2]) + "\n"
    if state[0] and state[1] and not state[2]:
        if len(state[0])>len(state[1]):
            string = string + str(state[0][0]) + "\n"
            string = string + str(state[0][1]) + " " +str(state[1][0]) + "\n"
        else:
            string = string + "  " + str(state[1][0]) + "\n"
            string = string + str(state[0][0]) + " " +str(state[1][1]) + "\n"
    if not state[0] and state[1] and state[2]:
        if len(state[1])>len(state[2]):
            string = string + "  " + str(state[1][0]) + "\n"
            string = string + "  " + str(state[1][1]) + " " +str(state[2][0]) + "\n"
        else:
            string = string + "    " + str(state[2][0]) + "\n"
            string = string + "  " + str(state[1][0]) + " " +str(state[2][1]) + "\n"
    if state[0] and not state[1] and state[2]:
        if len(state[0])>len(state[2]):
            string = string + str(state[0][0]) + "\n"
            string = string + str(state[0][1]) + "    " +str(state[2][0]) + "\n"
        else:
            string = string + "    " + str(state[2][0]) + "\n"
            string = string + str(state[0][0]) + "   " +str(state[2][1]) + "\n"
                
    string = string + "------"
    print(string)
        

In [509]:
def validMoves(state):

    moves = []
    if state[0]:
        if state[1]:
            if state[0][0]<state[1][0]:
                moves.append([1,2])
        else:
            moves.append([1,2])
        
        if state[2]:
            if state[0][0]<state[2][0]:
                moves.append([1,3])
        else:
            moves.append([1,3])
    
    if state[1]:
        if state[0]:
            if state[1][0]<state[0][0]:
                moves.append([2,1])
        else:
            moves.append([2,1])
        
        if state[2]:
            if state[1][0]<state[2][0]:
                moves.append([2,3])
        else:
            moves.append([2,3])
            
    if state[2]:
        if state[1]:
            if state[2][0]<state[1][0]:
                moves.append([3,2])
        else:
            moves.append([3,2])
        
        if state[0]:
            if state[2][0]<state[0][0]:
                moves.append([3,1])
        else:
            moves.append([3,1])
            
    return moves
        
        

In [510]:
def makeMove(state, move):
    ring1,ring2 = move
    clone = copy.deepcopy(state)
    clone[ring2-1].insert(0,clone[ring1-1].pop(0))
    return clone

In [741]:
def trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF):
    Q={}
    outcomes = np.zeros(nRepetitions)
    epsilons = np.zeros(nRepetitions)
    showMoves = False
    
    for nGames in range(nRepetitions):
        epsilonDecayFactor *= epsilonDecayFactor
        epsilons[nGames] = epsilonDecayFactor
        step = 0
        board = [[1,2,3],[],[]]
        done = False
    
        while not done:        
            step += 1
            # making a move
            move = epsilonGreedy(epsilonDecayFactor, Q, board)
            newBoard = makeMoveF(board,move)
            if (stateMoveTuple(board,move)) not in Q:
                Q[stateMoveTuple(board,move)] = -1  # initial Q value for new board,move
                
            if winner(newBoard):
                # in winning state
                Q[stateMoveTuple(board,move)] = -1
                done = True
                outcomes[nGames] = step
            
            if step > 1:
                Q[stateMoveTuple(boardOld,moveOld)] = Q[stateMoveTuple(boardOld,moveOld)] + learningRate * (-1 + Q[stateMoveTuple(board,move)] - Q[stateMoveTuple(boardOld,moveOld)]) 
                
            boardOld, moveOld = board, move # remember board and move to Q(board,move) can be updated after next steps
            board = newBoard
    return Q,outcomes
    

In [801]:
def testQ(Q, maxSteps, validMovesF, makeMoveF):
    game = [[1,2,3],[],[]]
    retVal= [game]
    for step in range(maxSteps):
        moves = validMoves(game)
        Qs = np.array([Q.get((stateMoveTuple(board,m)), 0) for m in moves]) 
        move = moves[ np.argmax(Qs) ]
        game = makeMoveF(game,move)
        retVal.append(game)
        if winner(game):
            return retVal
    

In [793]:
def epsilonGreedy(epsilon, Q, board):
    moves = validMoves(board)
    index = np.random.randint(len(moves))
    if np.random.uniform() < epsilon:
        # Random Move
        return moves[index]
    else:
        # Greedy Move
        Qs = np.array([Q.get((stateMoveTuple(board,m)), 0) for m in moves]) 
        return moves[ np.argmax(Qs) ]

In [794]:
def stateMoveTuple(state, move):
    tuplecopy = state[:]
    tuplecopy = tuple(state[0]),tuple(state[1]),tuple(state[2])
    return tuple(tuplecopy),tuple(move)

In [795]:
def winner(board):
    if board == [[],[],[1,2,3]]:
        return True
    return False

# Examples

In [796]:
Q, stepsToGoal = trainQ(50, 0.5, 0.7, validMoves, makeMove)

In [797]:
stepsToGoal

array([  56.,  104.,   25.,   19.,   70.,    9.,   52.,   10.,   19.,
         17.,   59.,    9.,   15.,    9.,   20.,    9.,   25.,   11.,
         12.,   11.,    7.,    8.,    7.,    7.,    7.,    7.,    7.,
         29.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,
          7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,
          7.,    7.,    7.,    7.,    7.])

In [798]:
newstate = makeMove(state, move)
newstate

[[], [1, 3], [2]]

In [799]:
printState(newstate)

  1
  3 2
------


In [802]:
path = testQ(Q, 20, validMoves, makeMove)

In [803]:
path

In [10]:
for s in path:
    printState(s)
    print()

1     
2     
3     
------


2     
3   1 
------


3 2 1 
------


  1   
3 2   
------


  1   
  2 3 
------


1 2 3 
------


    2 
1   3 
------


    1 
    2 
    3 
------




## Grading

Download and extract `A5grader.py` from [A5grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A5grader.tar).

In [13]:
%run -i A5grader.py


Testing validMoves([[1], [2], [3]])

--- 10/10 points. Correctly returned [[1, 2], [1, 3], [2, 3]]

Testing validMoves([[], [], [1, 2, 3]])

--- 10/10 points. Correctly returned [[3, 1], [3, 2]]

Testing makeMove([[], [], [1, 2, 3]], [3, 2])

--- 10/10 points. Correctly returned [[], [1], [2, 3]]

Testing makeMove([[2], [3], [1]], [1, 2])

--- 10/10 points. Correctly returned [[], [2, 3], [1]]

Testing   Q, steps = trainQ(1000, 0.5, 0.7, validMoves, makeMove).

--- 10/10 points. Q dictionary has correct number of entries.

--- 10/10 points. The mean of the number of steps is 7.402 which is correct.

Testing   path = testQ(Q, 20, validMoves, makeMove).

--- 20/20 points. Correctly returns path of length 8, less than 10.

assign5 Execution Grade is 80/80

 Remaining 20 points will be based on your text describing the trainQ and test! functions.

assign5 FINAL GRADE is __/100


## Extra Credit

Modify your code to solve the Towers of Hanoi puzzle with 4 disks instead of 3.  Name your functions

    - printState_4disk
    - validMoves_4disk
    - makeMove_4disk

Find values for number of repetitions, learning rate, and epsilon decay factor for which trainQ learns a Q function that testQ can use to find the shortest solution path.  Include the output from the successful calls to trainQ and testQ.